In [8]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from implementations import * 
from proj1_helpers import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# import some data to test the functions
from helpers import *

ids_tr,predictions_tr,data_tr, ids_te,data_te = load_boson_data()

count_s = 0
count_b = 0
    

    
print(data_tr.shape)

predictions_tr,data_tr = equalize_predictions(predictions_tr,data_tr)

print(data_tr.shape)

            
y = predictions_tr

x, mean_x, std_x = standardize(data_tr)

y,tx = build_model_data(x,y)

y.shape, x.shape, tx.shape, data_tr.shape

(99, 30)
(72, 30)


ValueError: too many values to unpack (expected 2)

In [ ]:
# gradient descent
initial_w = np.zeros(tx.shape[1])
max_iters = 20
gamma = 0.1
loss, w = gradient_descent(y, tx, initial_w, max_iters, gamma, print_output=False, plot_losses=True)
loss,w

In [ ]:
# stochastic gradient descent
# gradient descent
initial_w = np.array([0, 0])
max_iters = 20
gamma = 0.5
loss, w = gradient_descent(y, tx, initial_w, max_iters, gamma, batch_size=1, print_output=False, plot_losses=True)
loss,w

In [ ]:
# analityc solution
least_squares(y, tx)

In [ ]:
# ridge regression to choose smaller weights (the simpler the model the better)
lambda_ = 0.02 # how to properly choose lambda?
ridge_regression(y, tx, lambda_)

In [ ]:
# logistic regression (it is gradient descent with a different loss function)
from helpers import sample_data, load_data
from plots import visualization

# load data.
height, weight, gender = load_data()

# build sampled x and y.
seed = 1
y = np.expand_dims(gender, axis=1)
X = np.c_[height.reshape(-1), weight.reshape(-1)]
y, X = sample_data(y, X, seed, size_samples=200)
x, mean_x, std_x = standardize(X)

tx = build_poly(x, 1)
initial_w = np.ones(tx.shape[1])
max_iters = 50
gamma = 0.2
minloss, w = gradient_descent(y, tx, initial_w, max_iters, gamma, batch_size=-1, print_output=False, plot_losses=True, costfunc=CostFunction.PROB)
# w = np.array(w).reshape(-1, 1)
visualization(y, x, mean_x, std_x, w, "classification_by_least_square")
minloss, w

In [ ]:
tes_std, _, _ = standardize(data_te)
tx_std =  build_poly(tes_std, 1)

print(tx_std.shape)
predict_labels(w, tx_std)

## 0. Imports

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from implementations import * 
from proj1_helpers import *
%load_ext autoreload
%autoreload 2

## 1. Load data

In [ ]:
# just load the train data set
data_path = "../dataset/train.csv"
y, x_raw, col_labels = load_csv_data(data_path, sub_sample=False)
y.shape, x_raw.shape

## 2. fill with nan values where there are invalid entries 

In [ ]:
# example
k = np.array([
    [1,     0, -999],
    [7,     2,   -5],
    [-7,    0,    5],
    [-999, -1,    5],
    [-7,    2, -999]
])

print(k)
print()

# fill
nan_values = [-999, 0]
k = fill_with_nan_list(k, nan_values)
print(k)
print()

# ->>>> NB: probably better to substitute after standardization at step 5 <<<<-

# generate meaningful values with: 
# - np.nanmean(x, axis=0) (mean of each colum without considering nan values)
# - np.nanmedian(x, axis=0) (mean of each colum without considering nan values)
# - np.nanstd(x, axis=0) (mean of each colum without considering nan values)
k = sustitute_nans(k, substitutions=[-100, -200, -300]) 
# e.g. x = sustitute_nans(x, substitutions=np.nanmean(x, axis=0)) 
print(k)

In [ ]:
x = fill_with_nan_list(x_raw, nan_values=[0, -999])
x.shape

## 3. standardize (it won't consider the np.nan entries)

In [ ]:
x, mean_x, std_x = standardize(x)
x.shape

## 4. either drop_nan_rows/column or substitute nan values with something meaningful

In [ ]:
# check drop_nan_rows/columns and sustitute_nans in implementations.py
# np.nanmean(x, axis=0) should be an array of 0s (the column are standardized)
x = sustitute_nans(x, substitutions=np.nanmean(x, axis=0)) 
x.shape

## 5. train and test models:

### Ridge regression with cross validation

In [ ]:
# just show how the function ratios_visualization works

model1 = np.array([
    [0.65, 0.9], # relative to degrees[0]
    [0.7, 0.8],  # relative to degrees[1]
    [0.7, 0.7]]) # relative to degrees[2]
model2 =  np.array([
    [0.7, 0.6],  # relative to degrees[0]
    [0.9, 1],    # relative to degrees[1]
    [0.8, 0.8]]) # relative to degrees[2]
degree_list = np.array([2, 4, 7])
lambdas = np.array([0.1, 0.5]) # x asis
 
ratios_visualization([model1, model2, model1-0.1], degree_list, lambdas, x_label="x label", log_axis_x=False, 
                     save_figure_with_name="test_figure")

In [ ]:
# num_sets: number of sets in which the dataset will be splitted to run the cross validation
# degree_list: list of degree to be tried
# lambdas: list of lambdas to be tried 
def cross_validation_ridge_regression(x, y, num_K_sets, degree_list, lambdas):
    seed = 2
    
    # split indices in k sets
    k_indices = build_k_indices(y, num_K_sets, seed)
    
    ratio_tr = [] # matrix success ratio obtained with the training sets
    ratio_te = [] # matrix success ratio obtained with the test sets

    # define lists to store the loss of training data and test data
    for nfigure, degree in enumerate(degree_list): # one figure per degree
        tx = build_poly(x, degree) 
        
        # one row (figure) per degree
        ratio_tr.append([])
        ratio_te.append([])
        
        for npoint, lambda_ in enumerate(lambdas):
            # for each lambda we compute the expected ratio of success (this will be the a point in the figure)
            ratio_tr[nfigure].append(0)
            ratio_te[nfigure].append(0)
            
            for k_curr in range(num_K_sets):
                train, test = get_kth_set(y, tx, k_indices, k_curr)
                
                # train the model, just line should change depending on the chosen training 
                _, w = ridge_regression(train.y, train.tx, lambda_)
                
#                 # gradient descent
#                 initial_w = np.zeros(tx.shape[1])
#                 max_iters = 50
#                 gamma = lambda_
#                 _, w = gradient_descent(train.y, train.tx, initial_w, max_iters, gamma, batch_size=-1, print_output=False, plot_losses=False, costfunc=CostFunction.MSE)
                
                # compute how good is the model
                ratio_tr[nfigure][npoint] += compute_loss(train.y, train.tx, w, costfunc=CostFunction.SUCCESS_RATIO)
                ratio_te[nfigure][npoint] += compute_loss(test.y, test.tx, w, costfunc=CostFunction.SUCCESS_RATIO)
            
            # average the ratio obtained with the cross validation
            ratio_tr[nfigure][npoint] /= num_K_sets
            ratio_te[nfigure][npoint] /= num_K_sets
            
    ratio_tr = np.array(ratio_tr)
    ratio_te = np.array(ratio_te)
    
#     print(ratio_tr.shape) # #degree x #lambdas
#     print(ratio_te.shape)
#     print(degrees.shape)
#     print(lambdas.shape)
    
    return ratio_tr, ratio_te

In [ ]:
# the lambda of the ridge regression does not affect much the model
nlambdas = 3
lambdas = np.linspace(0, 1000, nlambdas)

num_K_sets = 5
degree_list = [2, 3, 4]

ratio_tr, ratio_te = cross_validation_ridge_regression(x, y, num_K_sets, degree_list, lambdas)

In [ ]:
# there is A LOT of data => there is no overfitting => the training error is close to the test error => it does
# not makes sense to do the cross validation (just use the whole set to train the data and compute the 
# SUCCESS_RATIO on the training data)
cross_validation_visualization(ratio_tr, ratio_te, degree_list, lambdas, "lambda (ridge_regression)")

ratio_tr.shape, ratio_te.shape

### Ridge regression without cross validation

In [ ]:
# num_sets: number of sets in which the dataset will be splitted to run the cross validation
# degree_list: list of degree to be tried
# lambdas: list of lambdas to be tried 
def ridge_regression_tuning(x, y, degree_list, lambdas):    
    
    ratios = [] # matrix success ratio obtained with the training sets

    # define lists to store the loss of training data and test data
    for nfigure, degree in enumerate(degree_list): # one figure per degree
        tx = build_poly(x, degree) 
        
        # one row (figure) per degree
        ratios.append([])
        
        for npoint, lambda_ in enumerate(lambdas):
            # for each lambda we compute the expected ratio of success (this will be the a point in the figure)
            ratios[nfigure].append(0)
            
            # train the model, just line should change depending on the chosen training 
            _, w = ridge_regression(y, tx, lambda_)

            ratios[nfigure][npoint] = compute_loss(y, tx, w, costfunc=CostFunction.SUCCESS_RATIO)
            
    ratios = np.array(ratios)
    
    return ratios

In [ ]:
# the lambda of the ridge regression does not affect mush the model
nlambdas = 3
lambdas = np.linspace(0, 1000, nlambdas)
degree_list = [2, 3, 4]

ratios = ridge_regression_tuning(x, y, degree_list, lambdas)

In [ ]:
x_axis = lambdas
figure_name = "ridge regression,nan_values=[0, -999],substitutions=nanmean(x, axis=0)"
ratios_visualization([ratios], degree_list, x_axis, x_label="lambas", log_axis_x=False,
                    save_figure_with_name=figure_name)

### Gradient descent try

In [ ]:
degree = 4
tx = build_poly(x, degree) 

### Logistic regression

In [ ]:
initial_w = np.zeros(tx.shape[1])-1
max_iters = 5 # try a small one first to check it is converging 
gamma = 0.000000000002 # very small (for higher diverges)

_, w = gradient_descent(y, tx, initial_w, max_iters, gamma, batch_size=-1, 
                        print_output_with_weights=[0, 1, 2], plot_losses=True, costfunc=CostFunction.MSE)

print(compute_loss(y, tx, w, costfunc=CostFunction.SUCCESS_RATIO))